### Домашнее задание
1. Использовать dataset MovieLens.
2. Посторить рекомендации (регрессия,предсказываем оценку) на фичах:  
* TF-IDF на тегах и жанрах
* Средние оценки(+median, variance , etc.)пользователя и фильма
3. Оценить RMSE на тестовой выборке.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from datetime import datetime
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [19]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [4]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [82]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [81]:
tags["tag"].count()

3683

In [4]:
# соединяю таблицу с жанрами и тегами фильмов

join_genres_tags = movies.join(tags.set_index('movieId'), on = 'movieId')
join_genres_tags

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN,NaN,NaN
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN,NaN,NaN
9739,193585,Flint (2017),Drama,NaN,NaN,NaN
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN,NaN,NaN


In [102]:
join_genres_tags[join_genres_tags.title == 'Forrest Gump (1994)']

,movieId,title,genres,userId,tag,timestamp
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,474.0,shrimp,1.137376e+09
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,474.0,Vietnam,1.137376e+09
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,533.0,bubba gump shrimp,1.424754e+09
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,533.0,lieutenant dan,1.424754e+09
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,533.0,stupid is as stupid does,1.424754e+09
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,567.0,bittersweet,1.525288e+09
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,567.0,emotional,1.525288e+09
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,567.0,heartwarming,1.525288e+09
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,567.0,touching,1.525288e+09


In [94]:
join_genres_tags.tag.unique()

array(['pixar', 'fun', 'fantasy', ..., 'star wars', 'gintama', 'remaster'],
      dtype=object)

In [5]:
tag_genres = join_genres_tags.groupby('tag')['genres'].count().sort_values(ascending = False)
tag_genres.head(15)

tag
In Netflix queue     131
atmospheric           36
thought-provoking     24
superhero             24
Disney                23
surreal               23
funny                 23
religion              22
dark comedy           21
sci-fi                21
quirky                21
psychology            21
suspense              20
twist ending          19
crime                 19
Name: genres, dtype: int64

In [6]:
#функция разбития жанров(через пробел)

def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [7]:
movie_genres = [change_string(g) for g in movies.genres.values]

In [8]:
join_genres_tags.dropna(inplace = True)

In [9]:
tag_strings = []
movies = []

for movie, group in tqdm(join_genres_tags.groupby('title')):
    tag_strings.append(' '.join([str(s).replace(' ', '').replace('-', '') for s in group.tag.values]))
    movies.append(movie)

  0%|          | 0/1572 [00:00<?, ?it/s]

In [10]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(tag_strings)

In [11]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [12]:
neigh = NearestNeighbors(n_neighbors = 15, n_jobs = -1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=15)

In [13]:
for i in range(len(movies)):
    if 'Forrest Gump (1994)' == movies[i]:
        print(i)

466


In [14]:
tag_strings[466]

'shrimp Vietnam bubbagumpshrimp lieutenantdan stupidisasstupiddoes bittersweet emotional heartwarming touching'

In [15]:
test = change_string("bittersweet|shrimp ")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance = True)

In [16]:
res

(array([[0.88514579, 1.        , 1.        , 1.02594601, 1.05144041,
         1.10903841, 1.25553314, 1.27108951, 1.29742592, 1.30640533,
         1.31216091, 1.32546209, 1.35369858, 1.41421356, 1.41421356]]),
 array([[1457,  822,  661,  466,  748, 1012,   24,  778,  847, 1093, 1465,
          665,  407, 1091,  268]], dtype=int64))

In [17]:
for i in res[1][0]:
    print(movies[i])

Two Days, One Night (Deux jours, une nuit) (2014)
Magnolia (1999)
In a Lonely Place (1950)
Forrest Gump (1994)
La La Land (2016)
Pan's Labyrinth (Laberinto del fauno, El) (2006)
A.I. Artificial Intelligence (2001)
Life Is Beautiful (La Vita è bella) (1997)
Mary and Max (2009)
Punch-Drunk Love (2002)
Up (2009)
In the Mood For Love (Fa yeung nin wa) (2000)
Eternal Sunshine of the Spotless Mind (2004)
Pulp Fiction (1994)
City of God (Cidade de Deus) (2002)


In [20]:
movies.iloc[res[1][0]]

,movieId,title,genres
1457,1980,Friday the 13th Part VII: The New Blood (1988),Horror
822,1082,"Candidate, The (1972)",Drama
661,866,Bound (1996),Crime|Drama|Romance|Thriller
466,533,"Shadow, The (1994)",Action|Adventure|Fantasy|Mystery
748,981,Dangerous Ground (1997),Drama
1012,1323,Amityville 3-D (1983),Horror
24,25,Leaving Las Vegas (1995),Drama|Romance
778,1020,Cool Runnings (1993),Comedy
847,1116,"Single Girl, A (Fille seule, La) (1995)",Drama
1093,1419,Walkabout (1971),Adventure|Drama


In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
lr = LinearRegression()

In [23]:
X_train_tfidf

<1572x1472 sparse matrix of type '<class 'numpy.float64'>'
	with 3598 stored elements in Compressed Sparse Row format>

In [24]:
movies_new = movies.copy()

In [25]:
movies_new["genres"] =  [change_string(g) for g in movies.genres.values]

In [26]:
movies_new.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [27]:
tfidf_transformer = TfidfVectorizer()
tfidf_transformer.fit(movies_new["genres"])

TfidfVectorizer()

In [28]:
mov_with_rat = movies_new.merge(ratings)
mov_with_rat

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action Animation Comedy Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation Comedy Fantasy,184,3.5,1537109545
100833,193585,Flint (2017),Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action Animation,184,3.5,1537110021


In [29]:
model_1 = mov_with_rat[mov_with_rat["userId"] == 111]
model_1

,movieId,title,genres,userId,rating,timestamp
394,5,Father of the Bride Part II (1995),Comedy,111,3.5,1517441319
886,16,Casino (1995),Crime Drama,111,4.5,1518640768
1289,24,Powder (1995),Drama SciFi,111,2.5,1518640854
1701,34,Babe (1995),Children Drama,111,2.5,1516153818
1899,39,Clueless (1995),Comedy Romance,111,4.0,1516153474
...,...,...,...,...,...,...
100732,183197,Dave Chappelle: Equanimity (2017),Comedy,111,3.5,1517440660
100734,183227,Dave Chappelle: The Bird Revelation (2017),Comedy,111,3.5,1517440658
100746,183959,Tom Segura: Disgraceful (2018),Comedy,111,4.5,1517440654
100747,184015,When We First Met (2018),Comedy,111,3.5,1518640587


In [30]:
X,y = tfidf_transformer.transform(model_1["genres"]), model_1["rating"]

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [33]:
lr.fit(X_train,y_train)

LinearRegression()

In [34]:
predict_model_1 = lr.predict(X_test)

In [35]:
predict_model_1

array([3.87636931, 3.28014107, 4.20021809, 3.44293833, 3.24934041,
       2.11265435, 3.44293833, 2.04585945, 3.3437312 , 3.66909119,
       3.77903093, 3.16216717, 3.44293833, 4.06289383, 3.49548629,
       3.45344931, 2.42577798, 3.50555157, 3.30546176, 3.44293833,
       3.44293833, 3.87636931, 3.30970942, 3.69425703, 3.44293833,
       3.44293833, 3.44293833, 3.23879595, 3.44293833, 3.1734536 ,
       3.50555157, 2.22311376, 3.77903093, 3.01015085, 2.96366019,
       3.28014107, 3.44293833, 3.44293833, 3.44293833, 2.57488377,
       3.44293833, 3.44293833, 3.14211234, 1.56184663, 1.7960871 ,
       4.4776956 , 3.23879595, 1.54904606, 3.28014107, 3.2411042 ,
       3.17190814, 3.36137332, 3.34952415, 2.98474884, 3.87636931,
       4.20021809, 3.84074258, 2.41678735, 1.84094636, 3.00828281,
       3.3423648 , 2.32570766, 3.44293833, 2.97118508, 3.44293833,
       2.05965776, 3.87636931, 3.44293833, 3.90972688, 3.44293833,
       3.77903093, 2.90389957, 4.6797388 , 3.28014107, 3.34373

In [161]:
y_test

86778     3.5
82569     4.0
99786     4.0
100660    4.0
32308     4.0
         ... 
57760     2.5
46728     3.0
100256    4.5
77350     3.5
2324      3.5
Name: rating, Length: 130, dtype: float64

In [162]:
lr.coef_

array([ 6.87341729e-01,  1.21820024e-03, -3.62910797e-01,  2.75023516e-01,
        9.72871401e-02,  3.98839640e-01,  9.00617096e-01,  2.69303282e-01,
       -1.83166645e-01,  0.00000000e+00, -1.88351583e+00, -3.62072073e-01,
        1.15192649e-01, -3.91038091e-01, -1.90962213e-01, -2.55792413e-01,
       -5.14996238e-01, -6.96732850e-01,  8.71163952e-01,  1.84015643e-01])

In [36]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [45]:
rmse = sqrt(mean_squared_error(y_test, predict_model_1))

In [46]:
rmse

0.9894966443590865

In [47]:
# нашел новую библиотеку - решил через нее проверить

In [39]:
pip install ml_metrics

  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-py3-none-any.whl size=7848 sha256=6175aa20420030e90efa98e6fac5724a582bd249a4a0f8c59936aaa99989c676
  Stored in directory: c:\users\sergey\appdata\local\pip\cache\wheels\09\2d\e7\58a96e635aedc543a3a78ff855dc9911d98fc1640e3921c823
Successfully built ml-metrics
Note: you may need to restart the kernel to use updated packages.


In [48]:
from ml_metrics import rmse

In [49]:
rmse(y_test, predict_model_1)

0.9894966443590865

In [178]:
# постарался вытащить разные данные из таблиц

In [74]:
# соединяю таблицу с рейтингами и названиями фильмов

join_ratings = ratings.join(movies.set_index('movieId'), on='movieId')

In [76]:
join_ratings

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Get Out (2017),Horror
100834,610,168252,5.0,1493846352,Logan (2017),Action|Sci-Fi


In [10]:
df_mov_rat = pd.merge(movies,ratings,on = 'movieId')
df_mov_rat.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [56]:
# df_mov_rat_gen = df_mov_rat.join(tags.set_index('movieId'), on ='movieId')

In [60]:
df_mov_rat.groupby('genres')['rating'].count().sort_values(ascending = False)

genres
Comedy                                    7196
Drama                                     6291
Comedy|Romance                            3967
Comedy|Drama|Romance                      3000
Comedy|Drama                              2851
                                          ... 
Crime|Drama|Film-Noir|Romance                1
Crime|Drama|Film-Noir|Romance|Thriller       1
Adventure|Fantasy|Musical|Romance            1
Crime|Drama|Horror|Mystery                   1
Comedy|Romance|Western                       1
Name: rating, Length: 951, dtype: int64

In [63]:
df_1 = df_mov_rat.groupby('genres')['rating'].count().sort_values(ascending = False)
df_1.head()

genres
Comedy                  7196
Drama                   6291
Comedy|Romance          3967
Comedy|Drama|Romance    3000
Comedy|Drama            2851
Name: rating, dtype: int64

In [ ]:
df_ = df_mov_rat.groupby('genres')['rating'].count().sort_values(ascending = False)
df_1.head()

In [12]:
# смотрю фильмы с максимальными рейтингами

df_mov_rat.groupby('title')['rating'].count().sort_values(ascending=False).head()

title
Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: rating, dtype: int64

In [17]:

df_rating_Count = pd.DataFrame(df_mov_rat.groupby('title', as_index = False)['rating'].count()
                              .rename(columns = {'rating' : 'rating_Count'}))
df_rating_Count.sort_values('rating_Count',ascending = False).head(10)

,title,rating_Count
3158,Forrest Gump (1994),329
7593,"Shawshank Redemption, The (1994)",317
6865,Pulp Fiction (1994),307
7680,"Silence of the Lambs, The (1991)",279
5512,"Matrix, The (1999)",278
8001,Star Wars: Episode IV - A New Hope (1977),251
4662,Jurassic Park (1993),238
1337,Braveheart (1995),237
8363,Terminator 2: Judgment Day (1991),224
7421,Schindler's List (1993),220


In [19]:
# create Dataframe: mean rating per movie
df_rating_Mean = pd.DataFrame(df_mov_rat.groupby('title', as_index = False)['rating'].mean()
                              .rename(columns={'rating' : 'rating_Mean'}))
df_rating_Mean.sort_values('rating_Mean',ascending=False).head()

,title,rating_Mean
3354,Gena the Crocodile (1969),5.0
8951,True Stories (1986),5.0
1986,Cosmic Scrat-tastrophe (2015),5.0
5269,Love and Pigeons (1985),5.0
7033,Red Sorghum (Hong gao liang) (1987),5.0


In [20]:
df_movies = pd.merge(df_rating_Count,df_rating_Mean, on = 'title')
df_movies.sort_values('rating_Count',ascending = False).head()

,title,rating_Count,rating_Mean
3158,Forrest Gump (1994),329,4.164134
7593,"Shawshank Redemption, The (1994)",317,4.429022
6865,Pulp Fiction (1994),307,4.197068
7680,"Silence of the Lambs, The (1991)",279,4.161290
5512,"Matrix, The (1999)",278,4.192446


In [22]:
df_movies.describe()

,rating_Count,rating_Mean
count,9719.000000,9719.000000
mean,10.375141,3.262388
std,22.406220,0.870004
min,1.000000,0.500000
25%,1.000000,2.800000
50%,3.000000,3.416667
75%,9.000000,3.910357
max,329.000000,5.000000


In [54]:
# добавляю жанры

df_mov_rat_gen = pd.merge(df_mov_rat,tags,on = 'userId')
df_mov_rat_gen.tail()

,movieId_x,title,genres,userId,rating,timestamp_x,movieId_y,tag,timestamp_y
4626861,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021,27156,psychology,1537094263
4626862,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021,193565,anime,1537098582
4626863,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021,193565,comedy,1537098587
4626864,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021,193565,gintama,1537098603
4626865,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021,193565,remaster,1537098592


In [25]:
df_rating= pd.DataFrame(ratings.groupby('rating', as_index = False)['userId'].count()
                         .rename(columns={'userId' : 'rating_Count'}))
df_rating.head()

,rating,rating_Count
0,0.5,1370
1,1.0,2811
2,1.5,1791
3,2.0,7551
4,2.5,5550


In [26]:
df_rating.describe()

,rating,rating_Count
count,10.000000,10.000000
mean,2.750000,10083.600000
std,1.513825,8335.389509
min,0.500000,1370.000000
25%,1.625000,3495.750000
50%,2.750000,8051.000000
75%,3.875000,13192.250000
max,5.000000,26818.000000


In [ ]:
# эксперименты понравишиеся при поиске решений:

In [30]:
data = pd.merge(movies,ratings)
data.head()


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [32]:
data = data.iloc[:1000000,:]
# lets make a pivot table in order to make rows are users and columns are movies. And values are rating
# пользователей в строки,фильмы в столбцы,значения - содержание таблицы

pivot_table = data.pivot_table(index = ["userId"],columns = ["title"],values = "rating")
pivot_table.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# ищу корреляцию с Forrest Gump (1994) 
    
movie_watched = pivot_table["Forrest Gump (1994)"]
similarity_with_other_movies = pivot_table.corrwith(movie_watched) 
similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending = False)
similarity_with_other_movies.head()

C:\Users\Sergey\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\Sergey\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


title
Lost & Found (1999)                          1.0
Cercle Rouge, Le (Red Circle, The) (1970)    1.0
Play Time (a.k.a. Playtime) (1967)           1.0
Killers (2010)                               1.0
Playing God (1997)                           1.0
dtype: float64